In [2]:
import pandas as pd
import random
from collections import defaultdict

shifts = {
    "00:00-08:00": range(0, 8),
    "05:00-13:00": range(5, 13),
    "08:00-16:00": range(8, 16),
    "12:00-20:00": range(12, 20),
    "14:00-22:00": range(14, 22),
    "16:00-24:00": range(16, 24)
}
# 读取数据
data = pd.read_csv('结果4.csv')

# 过滤特定分拣中心
sc60_data = data[data['分拣中心'] == 'SC60']

# 遗传算法参数
num_generations = 30
population_size = 50
num_workers = 200
num_shifts = 6
dates = sorted(sc60_data['日期'].unique())

# 染色体表示
def create_chromosome():
    return {
        (date, shift): {
            'regulars': random.sample(range(num_workers), k=random.randint(1, 10)),
            'temps': random.randint(1, 5)
        }
        for date in dates for shift in range(num_shifts)
    }

# 适应度函数
def calculate_fitness(chromosome):
    fitness = 0
    worker_days = defaultdict(int)
    for key in chromosome:
        day, shift = key
        info = chromosome[key]
        # 计算每个工人的出勤天数
        for worker in info['regulars']:
            worker_days[worker] += 1
        
        # 基本效率评估
        fitness += len(info['regulars']) * 1 + info['temps'] * 0.8
    
    # 惩罚连续工作超过7天的工人
    for worker, days in worker_days.items():
        if days > 7:
            fitness -= days  # 假设简单的惩罚

    # 确保没有工人的出勤率超过85%
    for worker, days in worker_days.items():
        if days / 30 > 0.85:
            fitness -= days  # 再次简单惩罚

    return fitness


# 选择
def select(population):
    sorted_population = sorted(population, key=calculate_fitness, reverse=True)
    return sorted_population[:len(population)//2]

# 交叉
def crossover(chromosome1, chromosome2):
    child = chromosome1.copy()
    crossover_point = random.randint(0, len(chromosome1))
    crossover_keys = list(chromosome1.keys())[crossover_point:]
    for key in crossover_keys:
        child[key] = chromosome2[key]
    return child

# 变异
def mutate(chromosome):
    mutation_key = random.choice(list(chromosome.keys()))
    chromosome[mutation_key]['regulars'] = random.sample(range(num_workers), k=random.randint(1, 10))
    chromosome[mutation_key]['temps'] = random.randint(1, 5)
    return chromosome

# 遗传算法主逻辑
# 遗传算法主逻辑，修改为返回最优染色体
def genetic_algorithm():
    population = [create_chromosome() for _ in range(population_size)]
    for generation in range(num_generations):
        population = select(population)
        new_generation = []
        while len(new_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)
            child = crossover(parent1, parent2)
            if random.random() < 0.04:
                child = mutate(child)
            new_generation.append(child)
        population = new_generation
    return max(population, key=calculate_fitness)  # 返回适应度最高的染色体


genetic_algorithm()


{('2023/12/01', 0): {'regulars': [96, 128, 70, 35, 33], 'temps': 5},
 ('2023/12/01', 1): {'regulars': [97, 173, 58, 145, 43, 12, 40, 33, 182, 19],
  'temps': 4},
 ('2023/12/01', 2): {'regulars': [173, 179, 120], 'temps': 5},
 ('2023/12/01', 3): {'regulars': [100, 199, 109, 47, 87, 62, 171, 147, 6, 119],
  'temps': 4},
 ('2023/12/01', 4): {'regulars': [6], 'temps': 3},
 ('2023/12/01', 5): {'regulars': [165, 69, 71, 161, 188, 138, 26, 106],
  'temps': 5},
 ('2023/12/02', 0): {'regulars': [0, 142, 188, 193, 113, 9, 122, 131],
  'temps': 4},
 ('2023/12/02', 1): {'regulars': [128, 164, 140, 38, 123], 'temps': 3},
 ('2023/12/02', 2): {'regulars': [187, 92, 145, 12, 14, 90, 44, 67, 22],
  'temps': 4},
 ('2023/12/02', 3): {'regulars': [101, 28, 134, 19, 142, 137, 62], 'temps': 5},
 ('2023/12/02', 4): {'regulars': [21, 70, 167, 116, 29], 'temps': 2},
 ('2023/12/02', 5): {'regulars': [102], 'temps': 2},
 ('2023/12/03', 0): {'regulars': [93], 'temps': 5},
 ('2023/12/03', 1): {'regulars': [128], '

In [3]:
# 获取最优染色体
best_chromosome = genetic_algorithm()

# 创建和填充结果DataFrame
results_df = pd.DataFrame([
    {'分拣中心': 'SC60', '日期': key[0], '班次': key[1], '出勤员工': f'正式工{worker}'}
    for key, value in best_chromosome.items() for worker in value['regulars']
] + [
    {'分拣中心': 'SC60', '日期': key[0], '班次': key[1], '出勤员工': f'临时工{value["temps"]}人'}
    for key, value in best_chromosome.items() if value['temps'] > 0
])

# 显示部分结果
results_df.head(20)


,分拣中心,日期,班次,出勤员工
0,SC60,2023/12/01,0,正式工188
1,SC60,2023/12/01,0,正式工32
2,SC60,2023/12/01,0,正式工11
3,SC60,2023/12/01,0,正式工90
4,SC60,2023/12/01,1,正式工169
5,SC60,2023/12/01,1,正式工189
6,SC60,2023/12/01,1,正式工102
7,SC60,2023/12/01,1,正式工108
8,SC60,2023/12/01,1,正式工35
9,SC60,2023/12/01,1,正式工123


In [4]:
import pandas as pd

# 获取最优染色体
best_chromosome = genetic_algorithm()

# 创建空的DataFrame用于存储输出结果
columns = ['分拣中心', '日期', '班次', '出勤员工']
results_df = pd.DataFrame(columns=columns)

# 遍历最优染色体中的结果，准备数据添加到DataFrame
rows = []  # 创建一个空列表用于存储即将添加的数据
for key, value in best_chromosome.items():
    date, shift_idx = key
    shift = list(shifts.keys())[shift_idx]  # 将班次索引转换为班次时间字符串
    # 添加正式工的出勤记录
    for worker in value['regulars']:
        rows.append({
            '分拣中心': 'SC60',
            '日期': date,
            '班次': shift,
            '出勤员工': f'正式工{worker}'
        })
    # 添加临时工的出勤记录
    if value['temps'] > 0:
        rows.append({
            '分拣中心': 'SC60',
            '日期': date,
            '班次': shift,
            '出勤员工': f'临时工{value["temps"]}人'
        })

# 使用pd.concat将所有行一次性添加到DataFrame
results_df = pd.concat([results_df, pd.DataFrame(rows)], ignore_index=True)

# 显示结果
results_df.head(20)


,分拣中心,日期,班次,出勤员工
0,SC60,2023/12/01,00:00-08:00,正式工11
1,SC60,2023/12/01,00:00-08:00,正式工21
2,SC60,2023/12/01,00:00-08:00,正式工190
3,SC60,2023/12/01,00:00-08:00,正式工12
4,SC60,2023/12/01,00:00-08:00,临时工4人
5,SC60,2023/12/01,05:00-13:00,正式工153
6,SC60,2023/12/01,05:00-13:00,正式工124
7,SC60,2023/12/01,05:00-13:00,临时工2人
8,SC60,2023/12/01,08:00-16:00,正式工178
9,SC60,2023/12/01,08:00-16:00,正式工199


In [5]:
# 使用pd.concat将所有行一次性添加到DataFrame
results_df = pd.concat([results_df, pd.DataFrame(rows)], ignore_index=True)

# 保存DataFrame到CSV文件
results_df.to_csv('结果6.csv', index=False, encoding='utf-8-sig')

# 打印消息确认文件已保存
print("数据已经成功保存到结果6.csv文件中。")


数据已经成功保存到结果6.csv文件中。


In [ ]:
import pandas as pd
import random
from collections import defaultdict

# 读取数据
data = pd.read_csv('结果4.csv')

# 过滤特定分拣中心
sc60_data = data[data['分拣中心'] == 'SC60']

# 定义班次时间范围
shifts = {
    "00:00-08:00": range(0, 8),
    "05:00-13:00": range(5, 13),
    "08:00-16:00": range(8, 16),
    "12:00-20:00": range(12, 20),
    "14:00-22:00": range(14, 22),
    "16:00-24:00": range(16, 24)
}

# 按日期和班次整理每小时数据
shift_data = {}
for shift, hours in shifts.items():
    mask = sc60_data['小时'].isin(hours)
    grouped = sc60_data.loc[mask].groupby('日期')['货量'].sum()
    shift_data[shift] = grouped.to_dict()

# 遗传算法参数
num_generations = 30
population_size = 50
num_workers = 200
num_shifts = 6
dates = sorted(sc60_data['日期'].unique())

# 染色体表示
# Chromosome Representation
def create_chromosome():
    return {
        (date, shift_key): {
            'regulars': random.sample(range(num_workers), k=random.randint(1, 10)),
            'temps': random.randint(1, 5)
        }
        for date in dates for shift_key in shifts.keys()
    }

# Calculate Fitness Function
def calculate_fitness(chromosome):
    fitness = 0
    worker_days = defaultdict(int)
    for (day, shift_key), info in chromosome.items():
        # Calculate each worker's attendance days
        for worker in info['regulars']:
            worker_days[worker] += 1

        # Base efficiency evaluation
        fitness += len(info['regulars']) * 1 + info['temps'] * 0.8
        
        # Check if the shift's cargo requirements are met
        required_output = shift_data[shift_key].get(day, 0)  # Use shift_key which matches shift_data keys
        if 25 * len(info['regulars']) + 20 * info['temps'] < required_output:
            fitness -= (required_output - (25 * len(info['regulars']) + 20 * info['temps'])) * 10  # Heavy penalty for not meeting requirements
    
    # Penalize workers who work more than 7 continuous days
    for worker, days in worker_days.items():
        if days > 7:
            fitness -= 20 * days  # Harsh penalty for working more than 7 days

    # Ensure no worker's attendance rate exceeds 85%
    for worker, days in worker_days.items():
        if days / 30 > 0.85:
            fitness -= 5 * days  # Penalty for exceeding 85% attendance

    return fitness


# 选择
def select(population):
    sorted_population = sorted(population, key=calculate_fitness, reverse=True)
    return sorted_population[:len(population)//2]

# 交叉
def crossover(chromosome1, chromosome2):
    child = chromosome1.copy()
    crossover_point = random.randint(0, len(chromosome1))
    crossover_keys = list(chromosome1.keys())[crossover_point:]
    for key in crossover_keys:
        child[key] = chromosome2[key]
    return child

# 变异
def mutate(chromosome):
    mutation_key = random.choice(list(chromosome.keys()))
    chromosome[mutation_key]['regulars'] = random.sample(range(num_workers), k=random.randint(1, 10))
    chromosome[mutation_key]['temps'] = random.randint(1, 5)
    return chromosome

# 遗传算法主逻辑
def genetic_algorithm():
    population = [create_chromosome() for _ in range(population_size)]
    for generation in range(num_generations):
        population = select(population)
        new_generation = []
        while len(new_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)
            child = crossover(parent1, parent2)
            if random.random() < 0.04:
                child = mutate(child)
            new_generation.append(child)
        population = new_generation
    return max(population, key=calculate_fitness)  # 返回适应度最高的染色体

genetic_algorithm()
